# 图骗预处理

In [95]:
import os, shutil, platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle 

%matplotlib inline 

## 工具函数:判断操作系统种类

In [96]:
def is_Windows_OS():
    if platform.system() == 'Windows':
        return True
    else:
        return False

## 定义的目录结构变量

In [97]:
# 训练集图片从zip包解压之后存放的目录
train_images_folder = 'train'

# 训练集图片目录
train_set_folder = 'train1'
train_set_folder_cat = 'train1/cat/'
train_set_folder_dog = 'train1/dog/'

# 验证集图片目录
validation_set_folder = 'valid'
validation_set_folder_cat = 'valid/cat/'
validation_set_folder_dog = 'valid/dog/'

# 测试集图片从zip包解压缩之后的存放目录
test_set_folder = 'test1'

## 新建有关目录结构

In [98]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir(train_set_folder)
os.mkdir(train_set_folder_cat)
os.mkdir(train_set_folder_dog)

rmrf_mkdir(validation_set_folder)
os.mkdir(validation_set_folder_cat)
os.mkdir(validation_set_folder_dog)

## 加载训练集目录

In [99]:
train_filenames = os.listdir(train_images_folder)
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

In [100]:
print (type(train_cat))
print (len(train_filenames))

<class 'filter'>
25000


In [101]:
print (train_filenames[24999])

dog.9999.jpg


定义一组移动有用的函数：

In [102]:
def move_cat_images():
    timer = 0
    for filename in train_cat:
        if timer < 3000:
            shutil.copyfile('train/'+filename, validation_set_folder_cat + filename)
        else:
            shutil.copyfile('train/'+filename, train_set_folder_cat + filename)
        timer = timer + 1

def move_dog_images():
    timer = 0
    for filename in train_dog:
        if timer < 3000:
            shutil.copyfile('train/'+filename, validation_set_folder_dog + filename)
        else:
            shutil.copyfile('train/'+filename, train_set_folder_dog + filename)
        timer = timer + 1
        
def create_cat_images_symlink():
    timer = 0
    for filename in train_cat:
        if timer <3000:
            os.symlink('train/'+filename, validation_set_folder_cat + filename)
        else:
            os.symlink('train/'+filename, train_set_folder_cat + filename)
        timer = timer + 1

def move_dog_images_symlink():
    timer = 0
    for filename in train_dog:
        if timer < 3000:
            os.symlink('train/'+filename, validation_set_folder_dog + filename)
        else:
            os.symlink('train/'+filename, train_set_folder_dog + filename)
        timer = timer + 1    

In [103]:
def fill_train_set_valid_set_folder_with_images():
    if is_Windows_OS():
        move_cat_images()
        move_dog_images()
    else:
        create_cat_images_symlink()
        move_dog_images_symlink()

In [104]:
fill_trainset_valid_set_folder()
print ('fill_train_set_valid_set_folder_with_images执行完毕')

fill_train_set_valid_set_folder_with_images执行完毕


## 预览测试集目录

In [8]:
test_filenames = os.listdir('test1')

In [9]:
print (len(test_filenames))

12500


In [10]:
file_name1 = "test1/" + test_filenames[0]
print (file_name1)

test1/9240.jpg


In [11]:
file_name2 = "test1/" + test_filenames[12499]
print (file_name2)

test1/6689.jpg


# 基准模型

我们选择一个简单的CNN模型作为基准模型，并且测试它执行图片分类任务的准确度。

## 首先导入所需类库：

## 搭建模型

## 训练模型

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
model = Sequential()

# Convolution
model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))

# Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# Second convolutional layer
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('train1',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

model.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_steps = 2000)

model_json = model.to_json()
with open("./model.json","w") as json_file:
  json_file.write(model_json)

model.save_weights("./model.h5")
print("saved model..! ready to go.")

Using TensorFlow backend.


Found 25000 images belonging to 2 classes.
Epoch 1/25
8000/8000 [==============================] - 5358s 670ms/step - loss: 7.9690 - acc: 0.5001
Epoch 2/25
 869/8000 [==>...........................] - ETA: 1:19:23 - loss: 8.0130 - acc: 0.4974

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5546/8000 [===================>..........] - ETA: 27:14 - loss: 7.9755 - acc: 0.4997

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

8000/8000 [==============================] - 5328s 666ms/step - loss: 7.9721 - acc: 0.4999
Epoch 6/25
2413/8000 [========>.....................] - ETA: 1:02:05 - loss: 7.9836 - acc: 0.4992

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7330/8000 [==========================>...] - ETA: 7:26 - loss: 7.9714 - acc: 0.5000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3543/8000 [============>.................] - ETA: 49:32 - loss: 7.9743 - acc: 0.4998